# 1.1 First NGSolve example

Let us solve the Poisson problem of finding $u$ satisfying 

$$
\begin{aligned}
-\Delta u & = f && \text { in  the unit square},
\\
u & = 0 && \text{ on the bottom and right parts of the boundary},
\\
\frac{\partial u }{\partial n } & = 0 
&& \text{ on the remaining  boundary parts}.
\end{aligned}
$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [1]:
import netgen.gui
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

importing NGSolve-6.2.2204
optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2204
Using Lapack
Including sparse direct solver Pardiso
Running parallel using 16 thread(s)


#### 2. Generate an unstructured mesh

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh.nv, mesh.ne   # number of vertices & elements 

 Generate Mesh from spline geometry


(37, 52)

* Here we prescribed a maximal mesh-size of 0.2 using the `maxh` flag. 

* The mesh can be viewed by switching to the `Mesh` tab in the Netgen GUI. 

#### 3. Declare a finite element space:

In [3]:
fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space

125

Python's help system displays further documentation.

In [ ]:
help(fes)

#### 4. Declare test function, trial function, and grid function 

* Test and trial function are symbolic objects - called `ProxyFunctions` -  that help you construct bilinear forms (and have no space to hold solutions). 

* `GridFunctions`, on the other hand, represent functions in the finite element space and contains memory to hold coefficient vectors.

In [4]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

Alternately, you can get both the trial and test variables at once:

In [5]:
u, v = fes.TnT()

#### 5. Define and assemble linear and bilinear forms:

In [6]:
a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += x*v*dx
f.Assemble()

You can examine the linear system in more detail:

In [ ]:
print(f.vec)

In [ ]:
print(a.mat)

#### 6. Solve the system:

In [7]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

Thank you for using NGSolve


The Dirichlet boundary condition constrains some degrees of freedom. The argument `fes.FreeDofs()` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:

In [ ]:
print(gfu.vec)

## Ways to interact with NGSolve

* A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.


* You can write an entire python module in a text editor and call python on the command line. (A script of the above is provided in `poisson.py`.)
    ```
    python3 poisson.py
    ```
  
* If you want the Netgen GUI, then use `netgen` on the command line:
    ```
    netgen poisson.py
    ```
  You can then ask for a python shell from the GUI's menu options (`Solve -> Python shell`).
  